![1](ATLASOD.gif)

## Pirma pamoka

Šioje pamokoje mes:
* Susipažinsime su dalelių fizikos CERN ROOT biblioteka
* Išmoksime įskaityti failą su ATLAS detektoriumi užfiksuotais susidūrimų duomenimis
* Atliksime imtis reikalingas studijuoti $Z\to\ell\ell$ procesą
* Apskaičiuosime leptonų sistemos keturmatį vektorių ir nupiešime masės spektros histogramą

Z bozonas skyla beveik akimirksniu, įmanoma užfiksuoti tik skilimo produktus

In [2]:
# Įkraunama ROOT biblioteka
import ROOT
%jsroot on

Welcome to JupyROOT 6.26/06


In [3]:
# Duomenų analizę pradedame pasitelkdami simuliacija. Atidarome ROOT failą
nuoroda = "https://atlas-opendata.web.cern.ch/atlas-opendata/samples/2020/2lep/MC/mc_363356.ZqqZll.2lep.root"
failas = ROOT.TFile.Open(nuoroda)
# Iš failo nuskaitome "medį" (tree) su susidūrimų duomenimis
medis = failas.Get("mini")

In [4]:
# Atliekame paprastas operacijas su nuskaitytu medžiu: išvedame susidūrimų skaičių į ekraną
n_viso = medis.GetEntries()
print(f"Faile rasta {n_viso} įvykių")

Faile rasta 1403146 įvykių


In [6]:
# Pažiūrėkime kokią informaciją galime rasti savo medyje
medis.Print()

******************************************************************************
*Tree    :mini      : mini                                                   *
*Entries :  1403146 : Total =      1461169448 bytes  File  Size =  380472501 *
*        :          : Tree compression factor =   3.84                       *
******************************************************************************
*Br    0 :runNumber : runNumber/I                                            *
*Entries :  1403146 : Total  Size=    5614273 bytes  File Size  =      28188 *
*Baskets :       12 : Basket Size=    1632768 bytes  Compression= 196.28     *
*............................................................................*
*Br    1 :eventNumber : eventNumber/I                                        *
*Entries :  1403146 : Total  Size=    5614309 bytes  File Size  =    3355513 *
*Baskets :       12 : Basket Size=    1632768 bytes  Compression=   1.65     *
*...................................................

### Kiekviena medžio "šaka" aprašo vis kitokį dalyką. Štai poros iš jų paaiškinimas:
`runNumber` - duomenų rinkimo periodo numeris  
`eventNumber` - unikalus įvykio identifikatorius  
`mcWeight` - simuliuoti įvykio svoriai reikalingi norint palyginti simuliuotus duomenis su tikrais duomenimis  
`lep_` - leptonų (elektronų ir muonų) duomenys. Pastebėkime, kad šie duomenys talpinami vektoriuje (`vector<float>`). Taip yra dėl to, kad kiekviename įvykyje gali susidaryti įvairus kiekis leptonų  
`jet_` - hadroninių čiurkšlių (jet) duomenys  
`photon` - fotonų duomenys  

In [7]:
# Visi įvykiai yra statistiškai nepriklausomi vienas nuo kito
# Pažiūrėkime, kaip atrodo pirmo įvykio leptonų skersinis impulsas (matuojami MeV)
# Naudojame for ciklą atlikti iteraciją, tačiau sustojame po pirmo įvykio 
# Kiekviena (simuliuota) dalelė turi savo PDG 
# Daugiau informacijos: https://particle.wiki/wiki/PDG_particle_numbering_scheme
for įvykis in medis:
    print(įvykis.lep_pt)
    break

{ 124174.f, 36407.2f }


In [15]:
# Kiekvienam įvykyje pabandome apskaičiuoti leptonų sistemos masę
# vienos-dimensijos, dvigubo tikslumo (H1D) histogramą leptonų sistemos masei

masės_histograma = ROOT.TH1D("masė", "masė", 100, 60, 120)

# Skaičiuojame, kiek įvykių pasirinkome (iš visų įmanomų)
n = 0

# Sukurkime po keturmatį vektorių abiems leptonų kandidatams
pirmas_leptonas = ROOT.TLorentzVector()
antras_leptonas = ROOT.TLorentzVector()

for įvykis in medis:
    # Įsitikiname, kad yra bent jau du leptonai
    # Kitaip praleidžiame šį įvykį
    if įvykis.lep_n < 2:
        continue
    # Leptonų krūviai privalo būti skirtingi
    if (įvykis.lep_charge[0] == įvykis.lep_charge[1]):
        continue 
        
    # Leptonų krūviai privalo būti skirtingi
    if (įvykis.lep_type[0] != įvykis.lep_type[1]):
        continue   

    # Apibrėžiame Lorenco vektorius
    # Priminimas: ATLAS ir CMS naudoja ne (px, py, pz, E) atskaitos sistemą o (pT, eta, phi, E)
    
    pirmo_pt = įvykis.lep_pt[0]/1000.
    antro_pt = įvykis.lep_pt[1]/1000.
    pirmo_E = įvykis.lep_E[0]/1000.
    antro_E = įvykis.lep_E[1]/1000.
    # Padalijome judesio kiekį (impulsą) ir energiją iš 1000 kad paversti vienetus iš MeV į GeV
    pirmo_phi = įvykis.lep_phi[0]
    antro_phi = įvykis.lep_phi[1]
    pirmo_eta = įvykis.lep_eta[0]
    antro_eta = įvykis.lep_eta[1]


    pirmas_leptonas.SetPtEtaPhiE(pirmo_pt, pirmo_eta, pirmo_phi, pirmo_E)
    antras_leptonas.SetPtEtaPhiE(antro_pt, antro_eta, antro_phi, antro_E)

    # Sudedami judesio kiekius gauname sistemos pilną (keturmatį) judesio kiekį p(Z) = p(l1) + p(l2)
    sistema = pirmas_leptonas + antras_leptonas
    
    # Kad gauti sistemos masę labai paprasta:
    masė = sistema.M()

    # Užpildome histogramą
    masės_histograma.Fill(masė)
    n += 1
    # Sustokime ties 10000 įvykių faile (kitaip gali ilgai užtrukti...)
    if n % 10000 == 0:
        print(f"Išanalizuota {n} įvykių. Baigiame")
        break

print(f"Iš viso išanalizuota {n}/{n_viso} įvykių")

Išanalizuota 10000 įvykių. Baigiame
Iš viso išanalizuota 10000/1403146 įvykių


Warning in <TDavixFile::Append>: Replacing existing TH1: masė (Potential memory leak).


In [16]:
# Sekantis etapas sukurti drobę (canvas)
drobė = ROOT.TCanvas()

In [19]:
# Drobė savaime būna tuščia, taigi nupiešiame ir histogramą
drobė.Draw()
masės_histograma.GetXaxis().SetTitle("Leptonų sistemos masė [GeV]")
masės_histograma.GetYaxis().SetTitle("Įvykiai / intervalo dydis")
masės_histograma.SetLineColor(ROOT.kRed)
masės_histograma.Draw()

Sveikiname, aiškiai matome Z bozono rezonansą (90 GeV).  
__Užduotis__: ar galite nupiešti dvi histogramas, vieną atskirai $Z\to\mu^+\mu^-$ skilimui, o kitą $Z\to e^+e^-$?  
_Patarimas_: antrą histogramą virš pirmos galima nupiešti pasinaudojus `SAME` komanda, pvz: `histograma2.Draw("SAME")`  
_Klausimas #1:_ Z bozona skyla į du miuonus ir du elektronus su vienoda tikimybe. Kodėl matomi truputį skirtingai skaičiai elektronų ir miuonų porų?